<h1>Extracting and Visualizing Stock Data</h1>


In [1]:
! pip install yfinance
! pip install pandas
! pip install requests
! pip install beautifulsoup4
! pip install plotly

In [2]:
! pip install --upgrade nbformat

In [3]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define Graphing Function


Define the function `make_graph`.It takes a dataframe with stock data, a dataframe with revenue data, and the name of the stock.


In [4]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

  
Using the `Ticker` function enter the ticker symbol of the stock to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [5]:
Tesla=yf.Ticker('TSLA')

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. 

In [6]:
Tesla_Info=Tesla.info
tesla_data=Tesla.history(period='max')
tesla_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3063 entries, 2010-06-29 to 2022-08-26
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          3063 non-null   float64
 1   High          3063 non-null   float64
 2   Low           3063 non-null   float64
 3   Close         3063 non-null   float64
 4   Volume        3063 non-null   int64  
 5   Dividends     3063 non-null   int64  
 6   Stock Splits  3063 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 191.4 KB


In [7]:
tesla_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29,1.266666,1.666666,1.169333,1.592666,281749140,0,0.0
2010-06-30,1.730666,2.027946,1.553333,1.588666,257915910,0,0.0
2010-07-01,1.666666,1.728000,1.351333,1.464000,123251835,0,0.0
2010-07-02,1.533333,1.540000,1.247333,1.280000,77036925,0,0.0
2010-07-06,1.333333,1.333333,1.055333,1.074000,102789510,0,0.0


  Reset the index

In [8]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,1.266666,1.666666,1.169333,1.592666,281749140,0,0.0
1,2010-06-30,1.730666,2.027946,1.553333,1.588666,257915910,0,0.0
2,2010-07-01,1.666666,1.728000,1.351333,1.464000,123251835,0,0.0
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77036925,0,0.0
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,102789510,0,0.0


Use the `requests` library to download the webpage [https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue](https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0220ENSkillsNetwork23455606-2022-01-01). Save the text of the response as a variable named `html_data`.


In [9]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

html_data  = requests.get(url).text

Parse the html data using `beautiful_soup`. Pay attention to the `html.parser`


In [10]:
soup=BeautifulSoup(html_data,'html.parser')
soup


<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" rel="canonical"/>
<title>Tesla Revenue 2010-2022 | TSLA | MacroTrends</title>
<meta content="Tesla annual/quarterly revenue history and growth rate from 2010 to 2022. Revenue can be defined as the amount of money a company receives from its customers in exchange for the sales of goods or services.  Revenue is the top line item on an income statement from which all costs and expenses are subtracted to arrive at net income.    
				
				&lt;ul style='margin-top:10px;'&gt;
				&lt;li&gt;Tesla revenue for the quarter ending June 30, 2022

Using `BeautifulSoup` or the `read_html` function extract the table with `Tesla Quarterly Revenue` and store it into a dataframe named `tesla_revenue`. 

In [11]:
df=pd.read_html(url)
tesla_revenue=df[1]
tesla_revenue.rename(columns={'Tesla Quarterly Revenue(Millions of US $)':'Date','Tesla Quarterly Revenue(Millions of US $).1':'Revenue'},inplace=True)
tesla_revenue

,Date,Revenue
0,2022-06-30,"$16,934"
1,2022-03-31,"$18,756"
2,2021-12-31,"$17,719"
3,2021-09-30,"$13,757"
4,2021-06-30,"$11,958"
5,2021-03-31,"$10,389"
6,2020-12-31,"$10,744"
7,2020-09-30,"$8,771"
8,2020-06-30,"$6,036"
9,2020-03-31,"$5,985"


Remove the comma and dollar sign from the `Revenue` column.


In [12]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

C:\Users\Robin\AppData\Local\Temp\ipykernel_6588\349343550.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")


Remove null or empty strings in the Revenue column.


In [13]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Display the last 5 row of the `tesla_revenue` 


In [14]:
tesla_revenue.tail(5)

,Date,Revenue
47,2010-09-30,31
48,2010-06-30,28
49,2010-03-31,21
51,2009-09-30,46
52,2009-06-30,27


Get the `Ticker` for GameStop and its ticker symbol is `GME`.


In [15]:
Gamestop=yf.Ticker('GME')

In [16]:
gme_data=Gamestop.history(period='max')

In [17]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19,1.666417,1.666417,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


Use the `requests` library to download the webpage <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html>. Save the text of the response as a variable named `html_data`.


In [18]:
url=' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data=requests.get(url).text

In [19]:
data=BeautifulSoup(html_data,'html.parser')
data

<!DOCTYPE html>

<!-- saved from url=(0105)https://web.archive.org/web/20200814131437/https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue -->
<html class="js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" style=""><!--<![endif]--><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script async="" src="./GameStop Revenue 2006-2020 _ GME _ MacroTrends_files/analytics.js.download" type="text/javascript"></script><script async="" src="./GameStop Revenue 2006-2020 _ GME _ MacroTrends_files/gpt.js.download" type="text/javascript"></script><script async="" src=".

In [20]:
table=pd.read_html('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html')
table[1]
gme_revenue=table[1].rename(columns={'GameStop Quarterly Revenue(Millions of US $)':'Date','GameStop Quarterly Revenue(Millions of US $).1':'Revenue'})
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")
gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue

C:\Users\Robin\AppData\Local\Temp\ipykernel_6588\768835914.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")


,Date,Revenue
0,2020-04-30,1021
1,2020-01-31,2194
2,2019-10-31,1439
3,2019-07-31,1286
4,2019-04-30,1548
...,...,...
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475


In [21]:
gme_revenue.tail(5)

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


Use the `make_graph` function to graph the Tesla Stock Data


In [22]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [23]:
make_graph(gme_data, gme_revenue, 'GameStop')